<a href="https://colab.research.google.com/github/2ovisa/AH2179/blob/main/project_finalcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stop ahead prediction
*the code was computed with the help from previous exercises, AI and https://towardsdatascience.com/6-methods-for-multi-step-forecasting-823cbde4127a/*



##Data Preparation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor


#-------------------------------------------------data preprocessing------------------------------------------------------------------

url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/ProjectAssignmentData/Dataset-PT.csv'
df = pd.read_csv(url, header = 1)
#print(df.head(30))
#df.info()
#print(df.columns.tolist())
#df = df.iloc[:1000]



In [ ]:
#Create unique trips
df = df.sort_values(['bus_id', 'Calendar_date', 'stop_sequence']).reset_index(drop=True)

df['trip_number'] = df.groupby(['bus_id','Calendar_date', 'stop_sequence']).cumcount()
df['unique_trip'] = (
    df['bus_id'].astype(str) + '_' +
    df['Calendar_date'].astype(str) + '_' +
    df['trip_number'].astype(str)
)

In [ ]:
#Reasure that all trips contains 27 stops
stops_ok = (
    df.groupby('unique_trip')['stop_sequence']
      .apply(lambda s: set(s.tolist()) == set(range(1,28)))
      .all()
)
print('Stops exakt 1..27 per trip:', stops_ok)

#Reasure that stop sequence is striclty increased by one step
mono_ok = (
    df.groupby('unique_trip')['stop_sequence']
      .apply(lambda s: (s.diff().fillna(1) == 1).all())
      .all()
)
print('Strikt +1 mellan rader inom trip:', mono_ok)

#Each of the horizons are looking at the right trip
for h in [1,3,6,12]:
    s_future = df.groupby('unique_trip')['stop_sequence'].shift(-h)
    ok = (s_future.dropna() - df.loc[s_future.notna(), 'stop_sequence'] == h).all()
    print(f'H={h} korrekt skift:', ok)

#the shift do not cross
for h in [1,3,6,12]:
    uid_future = df.groupby('unique_trip')['unique_trip'].shift(-h)
    cross_ok = (uid_future.dropna() == df.loc[uid_future.notna(), 'unique_trip']).all()
    print(f'H={h} ingen kors-trip:', cross_ok)


*Create the target values*

In [ ]:
#multi horizon targets

for h in [1,3,6,12]:
  df[f'arrival_delay_t+{h}'] = df.groupby('unique_trip')['arrival_delay'].shift(-h)

# ta bort rader som saknar alla  target
targets = [f"arrival_delay_t+{h}" for h in [1, 3, 6, 12]]
df = df[df[targets].notna().any(axis=1)].copy()

*Feature Engineering*

In [ ]:
#Normalizes the stop index
df["stop_sequence_norm"] = df.groupby("unique_trip")["stop_sequence"].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

#The amount of stops remaining in the route
df["stops_remaining"] = df.groupby("unique_trip")["stop_sequence"].transform(
    lambda x: x.max() - x
)

#Change in delay compared to previous stop
df["delay_diff"] = df.groupby("unique_trip")["arrival_delay"].diff().fillna(0)

#Moving Average of last 3 stops
df["delay_ma3"] = df.groupby("unique_trip")["arrival_delay"].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)
#Difference between current delay and moving average
df["delay_trend"] = df["arrival_delay"] - df["delay_ma3"]


###EDA

In [ ]:
#Descriptive statisrtcs
print(df["arrival_delay"].mean())
print(df["arrival_delay"].median())
#df["arrival_delay"].hist(bins=50)

*Plot of the distribution of arrival delay for each stop*

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x="stop_sequence", y="arrival_delay", alpha=0.3)

#mean trend line
sns.lineplot(data=df, x="stop_sequence", y="arrival_delay",
             estimator="mean", ci=None, color="red", linewidth=2, label="Average delay")

#reference line at 0 delay
plt.axhline(0, color="black", linestyle="--", alpha=0.7)

plt.xlabel("Stop sequence")
plt.ylabel("Arrival delay (s)")
plt.title("Arrival delay against stop sequence")
plt.legend()
plt.show()



*Correlations*

In [ ]:
#heatmap, arrival delays and dummies
corr = df[["arrival_delay","factor(weather)Snow", "factor(weather)Rain", "factor(weather)Normal", "factor(time_of_day)Morning_peak", "factor(time_of_day)Afternoon_peak", "factor(day_of_week)weekend", "factor(day_of_week)weekday","factor(temperature)Cold","factor(temperature)Extra_cold" ]].corr()

sns.heatmap(corr, annot=True, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# Correlation of each feature with arrival_delay

df_numeric = df.select_dtypes(include=['number'])

corr_matrix = df_numeric.corr()

arrival_corr = corr_matrix['arrival_delay'].sort_values(ascending=False)
print(arrival_corr)

##Data preprocessing

In [ ]:
unique_trips = df["unique_trip"].unique()
split_point = int(len(unique_trips) * 0.8)
train_trips = unique_trips[:split_point]
test_trips = unique_trips[split_point:]

#create a mask
train_mask = df["unique_trip"].isin(train_trips)
test_mask = ~train_mask



In [ ]:
#create a copy for visualization later
df_vis = df.copy()

drop_cols = [
    "Calendar_date", "bus_id", "route_id", "arrival_time",
    "unique_trip", "new_trip", "trip_number"
]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

#baseline dummies are removed to avoid multicollinearity
to_drop = ["factor(weather)Normal", "factor(temperature)Normal",
           "factor(day_of_week)weekend", "factor(time_of_day)Off-peak"]
df = pd.get_dummies(df, drop_first=False)

df = df.drop(columns=[c for c in to_drop if c in df.columns], errors="ignore")

#advanced its features are removed for the existing ITS
df_existing_its = df.copy()
drop_its = ['upstream_stop_delay',
'previous_bus_delay',
'previous_trip_travel_time',
'travel_time_for_previous_section',
'dwell_time',
'traffic_condition',
'recurrent_delay',
'delay_diff',
'delay_ma3',
'delay_trend']

df_existing_its = df_existing_its.drop(columns=[c for c in drop_its if c in df.columns])

*Existing ITS*

In [ ]:
#existing its
#targets
X_eits = df_existing_its.drop(["arrival_delay"] + targets, axis=1, errors="ignore")
y_eits = df_existing_its[targets]

#fill missing values
y_eits = y_eits.fillna(method='ffill').fillna(method='bfill')

X_train_eits, X_test_eits = X_eits[train_mask], X_eits[test_mask]
y_train_eits, y_test_eits = y_eits[train_mask], y_eits[test_mask]

scaler = StandardScaler()

X_train_eits = scaler.fit_transform(X_train_eits)
X_test_eits = scaler.transform(X_test_eits)

*Advanced ITS*

In [ ]:
#targets
X = df.drop(["arrival_delay"] + targets, axis=1, errors="ignore")
y = df[targets]

#missing values are filled
y = y.fillna(method='ffill').fillna(method='bfill')

In [ ]:
X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]

In [ ]:
#scale
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

*Randomized Search*

In [ ]:
#For XGBregressor
param_dist = {
    'n_estimators': [200, 400, 600, 800],
    'max_depth': [2, 4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9]
}

rand_search = RandomizedSearchCV(
    XGBRegressor(tree_method='hist', random_state=42),
    param_distributions=param_dist,
    n_iter=30,
    scoring='neg_mean_absolute_error',
    cv=3,
    n_jobs=-1
)

rand_search.fit(X_train, y_train)
print("Best params:", rand_search.best_params_)


In [ ]:
#for Random Forest Regressor
param_dist = {
    'n_estimators': [50, 80, 100, 200],
    'max_depth': [4, 8, 10, 15],
    'min_samples_split': [10, 12, 15],
    'min_samples_leaf': [2, 4, 6 ],
}

rand_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=0),
    param_distributions=param_dist,
    n_iter=3,
    scoring='neg_mean_absolute_error',
    cv=3,
    n_jobs=-1
)

rand_search.fit(X_train, y_train)
print("Best params:", rand_search.best_params_)

In [ ]:
#for K Neighbors
param_dist = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

rand_search = RandomizedSearchCV(
    KNeighborsRegressor(),
    param_distributions=param_dist,
    n_iter=3,
    scoring='neg_mean_absolute_error',
    cv=3,
    n_jobs=-1
)

rand_search.fit(X_train, y_train)
print("Best params:", rand_search.best_params_)

#Best params: {'weights': 'distance', 'p': 1, 'n_neighbors': 10

##Multi-output models

###*KNN*

In [ ]:
from sklearn.neighbors import KNeighborsRegressor


model = KNeighborsRegressor(n_neighbors=10, weights="distance", p=1)
model.fit(X_train, y_train)

#Predict on the test set
y_pred = model.predict(X_test)

###*Random Forest Regressor*

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(n_estimators=80, max_depth=15, random_state=42)
model.fit(X_train, y_train)

#Prediktion
y_pred = model.predict(X_test)

In [ ]:
#multioutput with wrapped
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

model = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=80, max_depth=15, random_state=42)
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

###*XGBoost*

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.5,
        random_state=42,
        tree_method='hist'
    )
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [ ]:
#multioutput with wrapped
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

xgb_model = MultiOutputRegressor(
    XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.5,
        random_state=42,
        tree_method='hist'
    )
)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)



##*Chained multi-output*

In [ ]:
#XGBoost
from sklearn.multioutput import RegressorChain
from xgboost import XGBRegressor


base_model = XGBRegressor(n_estimators=500,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.5,
        random_state=42,
        tree_method='hist'
    )

chain = RegressorChain(base_model, order =[0,1,2,3]) #0=t+1,1=t+3 osv
chain.fit(X_train, y_train)

y_pred = chain.predict(X_test)

In [ ]:
#Random forest
from sklearn.multioutput import RegressorChain
from sklearn.ensemble import RandomForestRegressor

base_model = RandomForestRegressor(n_estimators=80, max_depth=15, random_state=42)

chain = RegressorChain(base_model, order =[0,1,2,3]) #0=t+1,1=t+3 osv
chain.fit(X_train, y_train)

y_pred = chain.predict(X_test)


###*Existing ITS*

In [ ]:
from sklearn.multioutput import RegressorChain
from xgboost import XGBRegressor


#base_model_eits = XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=8,
#                          subsample=0.8, colsample_bytree=0.8, random_state=42,
#                          tree_method='hist')

base_model_eits = RandomForestRegressor(n_estimators=80, max_depth=15, random_state=42)

chain = RegressorChain(base_model_eits, order =[0,1,2,3]) #0=t+1,1=t+3 osv
chain.fit(X_train_eits, y_train_eits)

y_pred_eits = chain.predict(X_test_eits)

###*LSTM*
- *this is computed with the help of AI*

In [ ]:
import numpy as np

timesteps = 5
horizons = [1, 3, 6, 12]
features = ["arrival_delay"]

X_list, y_list = [], []

for trip_id, trip_data in df_vis.groupby("unique_trip"):
    arr = trip_data[features].values
    delays = trip_data["arrival_delay"].values

    for i in range(len(trip_data) - timesteps - max(horizons)):
        # input sequence
        X_list.append(arr[i:i+timesteps])

        # targets
        y_list.append([delays[i+timesteps+h-1] for h in horizons])

X = np.array(X_list)   # shape: (samples, timesteps, features)
y = np.array(y_list)   # shape: (samples, len(horizons))


In [ ]:
#TRAIN AND TEST SPLIT
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, X.shape[2])))
model.add(Dense(len(horizons)))   # en output per horisont
model.compile(optimizer="adam", loss="mae")

model.summary()


In [ ]:
#Train
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2
)


In [ ]:
#evaluate
y_pred = model.predict(X_test)

from sklearn.metrics import mean_absolute_error, r2_score

for i, h in enumerate(horizons):
    mae = mean_absolute_error(y_test[:, i], y_pred[:, i])
    r2  = r2_score(y_test[:, i], y_pred[:, i])
    print(f"Horizon t+{h}: MAE={mae:.2f}, R²={r2:.3f}")


##Evalutation

*Average delay per stop*

In [ ]:
#Shift the predictions back to get to their stop. This is done only for better visulisation
pred_df = pd.DataFrame(y_pred, columns=[f"pred_t+{h}" for h in [1,3,6,12]])
pred_df["stop_sequence"] = df_vis.loc[test_mask, "stop_sequence"].values
pred_df["unique_trip"] = df_vis.loc[test_mask, "unique_trip"].values
pred_df["arrival_delay"] = df_vis.loc[test_mask, "arrival_delay"].values

for h in [1,3,6,12]:
    pred_df[f"pred_t+{h}_aligned"] = (
        pred_df.groupby("unique_trip")[f"pred_t+{h}"].shift(h)
    )

In [ ]:
#average delay per stop
avg_aligned = pred_df.groupby("stop_sequence")[["arrival_delay"] +
    [f"pred_t+{h}_aligned" for h in [1,3,6,12]]].mean()

plt.figure(figsize=(10,6))
plt.plot(avg_aligned.index, avg_aligned["arrival_delay"], label="Actual", marker='o')
for h in [1,3,6,12]:
    plt.plot(avg_aligned.index, avg_aligned[f"pred_t+{h}_aligned"], '--', label=f"Predicted t+{h}")
plt.xlabel("Stop sequence")
plt.ylabel("Average Delay (s)")
plt.title("Average Delay Prediction Aligned per Stop (MultiOutputRegressor RF)")
plt.legend()
plt.grid(True)
plt.show()

*Feature importance*

In [ ]:
#Feature importance for RegressonChain
#collect feature importances per horizon
feature_names = list(X.columns)
importances = []
for i, est in enumerate(chain.estimators_):
    # base + previous predictions for later horizons
    extra = [f"y_pred_t+{[1,3,6,12][j]}" for j in range(i)]
    feats = feature_names + extra
    imp = pd.Series(est.feature_importances_, index=feats).sort_values(ascending=False)
    importances.append(imp)

#plot top features per horizon
plt.figure(figsize=(14, 10))
for i, (imp, h) in enumerate(zip(importances, [1,3,6,12])):
    plt.subplot(2, 2, i+1)
    top_imp = imp.head(10)
    top_imp[::-1].plot(kind="barh")  # reverse for readable order
    plt.title(f"Horizon t+{h}")
    plt.xlabel("Importance")
    plt.tight_layout()

plt.show()

In [ ]:
# Feature importance for MultiOutputRegressor
feature_names = list(X.columns)
importances = []

for i, est in enumerate(model.estimators_):  # one estimator per horizon
    imp = pd.Series(est.feature_importances_, index=feature_names).sort_values(ascending=False)
    importances.append(imp)

# plot top features per horizon
plt.figure(figsize=(14, 10))
for i, (imp, h) in enumerate(zip(importances, [1,3,6,12])):
    plt.subplot(2, 2, i+1)
    top_imp = imp.head(10)
    top_imp[::-1].plot(kind="barh")  # reverse for readability
    plt.title(f"Horizon t+{h}")
    plt.xlabel("Importance")
    plt.tight_layout()

plt.show()


In [ ]:
importances = model.feature_importances_

feat_imp = pd.Series(importances, index=X.columns).sort_values(ascending=False)

#top 5 features
print(feat_imp.head(5))

# Plot
plt.figure(figsize=(8,6))
feat_imp.head(10).plot(kind="barh")
plt.title("Top 10 Feature Importances (Shared across all horizons)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.gca().invert_yaxis()
plt.show()


*Metrics*

In [ ]:
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
r2 = [r2_score(y_test.iloc[:, i], y_pred[:, i]) for i in range(y_test.shape[1])]

for i, col in enumerate(y_test.columns):
    print(f"{col}: MAE={mae[i]:.2f}, R²={r2[i]:.3f}")


In [ ]:
#existing its
mae = mean_absolute_error(y_test_eits, y_pred_eits, multioutput='raw_values')
r2 = [r2_score(y_test_eits.iloc[:, i], y_pred_eits[:, i]) for i in range(y_test.shape[1])]

for i, col in enumerate(y_test_eits.columns):
    print(f"{col}: MAE={mae[i]:.2f}, R²={r2[i]:.3f}")